<a href="https://colab.research.google.com/github/sugarforever/wtf-langchain/blob/main/09_Callbacks/09_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Callback 示例

## 准备环境

1. 安装langchain版本0.1.0 ，以及openai

In [1]:
!pip install -q -U langchain==0.1.0  openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


2. 设置OPENAI API Key

In [2]:
# import os

# os.environ['OPENAI_API_KEY'] = "您的有效openai api key"

## 示例代码

1. 内置回调处理器 `StdOutCallbackHandler`

In [3]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("Who is {name}?")
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(name="Super Mario")

/Users/kennywu/opt/anaconda3/envs/aigc/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/kennywu/opt/anaconda3/envs/aigc/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Who is Super Mario?

> Finished chain.


'\n\nSuper Mario is a fictional character and the main protagonist of the popular video game franchise of the same name created by Nintendo. He is an Italian plumber who lives in the Mushroom Kingdom and goes on various adventures to rescue Princess Peach and save the kingdom from the evil Bowser. He is known for his iconic red hat, blue overalls, and mustache, and has become one of the most recognizable and beloved characters in the history of video games. '

2. 自定义回调处理器

我们来实现一个处理器，统计每次 `LLM` 交互的处理时间。

In [4]:
from langchain.callbacks.base import BaseCallbackHandler
import time

class TimerHandler(BaseCallbackHandler):

    def __init__(self) -> None:
        super().__init__()
        self.previous_ms = None
        self.durations = []

    def current_ms(self):
        return int(time.time() * 1000 + time.perf_counter() % 1 * 1000)

    def on_chain_start(self, serialized, inputs, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_chain_end(self, outputs, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration)

    def on_llm_start(self, serialized, prompts, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_llm_end(self, response, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration)

In [6]:
llm = OpenAI()
timerHandler = TimerHandler()
prompt = PromptTemplate.from_template("What is the HEX code of color {color_name}?")
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[timerHandler])
response = chain.run(color_name="blue")
print(response)
# response = chain.run(color_name="purple")
# print(response)

timerHandler.durations



The HEX code for the color blue is #0000FF.


[1048]

3. `Model` 与 `callbacks`

`callbacks` 可以在构造函数中指定，也可以在执行期间的函数调用中指定。

请参考如下代码：

In [7]:
timerHandler = TimerHandler()
llm = OpenAI(callbacks=[timerHandler])
response = llm.predict("What is the HEX code of color BLACK?")
print(response)

timerHandler.durations

/Users/kennywu/opt/anaconda3/envs/aigc/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The HEX code for black is #000000.


[1282]

In [8]:
timerHandler = TimerHandler()
llm = OpenAI()
response = llm.predict("What is the HEX code of color BLACK?", callbacks=[timerHandler])
print(response)

timerHandler.durations



The HEX code for black is #000000.


[22215]